In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
firefox_profile_dir = '/home/bird/.mozilla/firefox/iadzfbcv.default/'
[x for x in os.listdir(firefox_profile_dir) if x.endswith('sqlite')]

['cookies.sqlite',
 'storage-sync.sqlite',
 'kinto.sqlite',
 'storage.sqlite',
 'content-prefs.sqlite',
 'webappsstore.sqlite',
 'places.sqlite',
 'favicons.sqlite',
 'permissions.sqlite',
 'formhistory.sqlite']

In [3]:
storage_file = '{}/webappsstore.sqlite'.format(firefox_profile_dir)
storage_db = sqlite3.connect(storage_file)

In [4]:
# %load '/home/bird/Documents/tracking technologies/notebooks/get_sqlite_tables.py'
def list_tables_in_db(db):
    print(db.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())

list_tables_in_db(storage_db)

[('webappsstore2',)]


In [5]:
storage_df = pd.read_sql('SELECT * FROM webappsstore2', storage_db)[['originKey', 'key', 'value']]

In [6]:
storage_df['origin'] = storage_df.originKey.apply(lambda x: x[::-1].split(':.')[1])
storage_df = storage_df.drop('originKey', axis='columns')

In [7]:
storage_df.head()

,key,value,origin
0,yt-remote-device-id,"{""data"":""57597ada-8b1f-4ff6-9ad5-10f9fddba032""...",www.youtube.com
1,sidebarState-527273a994b0712768004193,"{""idRecentBoards"":[""58fa22b3e5648003501dcf80""]}",trello.com
2,ajs_user_traits,{},trello.com
3,debug,undefined,trello.com
4,ajs_group_properties,{},trello.com


In [8]:
cookies_file = '{}/cookies.sqlite'.format(firefox_profile_dir)
cookies_db = sqlite3.connect(cookies_file)
cookied_df = pd.read_sql('SELECT * FROM moz_cookies', cookies_db)
shared_values = []
for v in cookied_df.value.unique():
    matches = cookied_df[cookied_df.value.str.contains(v, regex=False)]
    if len(matches.baseDomain.unique()) > 5:
        shared_values.append(v)
potential_ids = [x for x in shared_values if(len(x) > 10) & ('com' not in x)] 

In [9]:
# From cookie table
print('We have', len(potential_ids), 'potential ids. Things like:')
potential_ids[0:5]

We have 25 potential ids. Things like:


['7d3a5a8f-16c6-4a00-b672-f68cb04d1141',
 'AABkok61JQUAABIntsWgWg',
 'A798FAB6-8B6A-45E1-BDC9-9132ED3416A5',
 'GGc421K1ICVt',
 '8a88bb49-9ef4-42d6-a720-70cadad54101']

In [10]:
for potential_id in potential_ids:
    if potential_id in storage_df.value.values:
        print(potential_id, 'found in local storage')

7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc found in local storage


In [11]:
repeated_id = '7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc'
storage_df[storage_df.value.str.contains(repeated_id)][['value', 'origin', 'key']]

,value,origin,key
27,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.latimes.com,taboola global:user-id
227,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.merriam-webster.com,taboola global:user-id
229,v2_bf1ca6db22b70291779ce41eb2e5aee5_7620423c-7...,www.merriam-webster.com,taboola-display-merriam-webster:session-data
232,v2_bf1ca6db22b70291779ce41eb2e5aee5_7620423c-7...,tpc.googlesyndication.com,taboola-display-merriam-webster:session-data
236,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,tpc.googlesyndication.com,taboola global:user-id
357,v2_9a6c86f52b3f9239067936a17472df9f_7620423c-7...,www.huffingtonpost.com,aol-huffingtonpost:session-data
359,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.huffingtonpost.com,taboola global:user-id
407,v2_66138b9f6bd0ec8912d19cb714efd912_7620423c-7...,www.latimes.com,latimes:session-data
984,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.bloomberg.com,taboola global:user-id
989,v2_96ea2f0533c59e2312c1f1112ced8f46_7620423c-7...,www.bloomberg.com,bloomberg:session-data


Can we create the undead?

* Create a completely fresh profile
* Copy across this local storage file
* Visit the latimes.com

[But does this really happen?](http://localhost:8888/notebooks/notebooks/evercookie_p2.ipynb)